In [20]:

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import echonet
import tqdm

In [24]:
if not os.path.exists('/workspace/data/drives/Local_SSD/sdc/Weights-20201103T193519Z-001/Weights'):
    os.system("cp -r /workspace/data/NAS/Weights-20201103T193519Z-001/ /workspace/data/drives/Local_SSD/sdc/")

In [25]:
mean = np.array([31.834011, 31.95879,  32.082172])
std = np.array([48.866325, 49.137333, 49.361984])
print(mean, std)

[31.834011 31.95879  32.082172] [48.866325 49.137333 49.361984]


In [26]:
# From https://github.com/echonet/dynamic/blob/master/scripts/InitializationNotebook.ipynb
data = "Example Analysis"
regression_model_checkpoint = "/workspace/data/drives/Local_SSD/sdc/Weights-20201103T193519Z-001/Weights/r2plus1d_18_32_2_pretrained.pt"

# Initialize and Run EF model

frames = 32
period = 2
batch_size = 20
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)


if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(regression_model_checkpoint)
    model.load_state_dict(checkpoint['state_dict'])
    

cuda is available, original weights


In [27]:
unique_firstrun =  "/Example Analysis"

kwargs = {"target_type": "Filename",
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period
          }

test_ds = echonet.datasets.Echo(split = "external_test", external_test_location = data, **kwargs, clips="all") # unique_fnames) #
#print(test_ds.split, test_ds.fnames)

#ds = echonet.datasets.Echo(root = "C:\\Datasets\\ReleasedData", **kwargs, split = "all", clips = "all")

test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size = 1, num_workers = 5, shuffle = False, pin_memory=(device.type == "cuda"))
# loss, yhat, y = echonet.utils.video.run_epoch(model, test_dataloader, train = False, optim = None, device = device, save_all=True, block_size = 20)

# with tqdm.tqdm(total = len(test_dataloader)) as pbar:
#     for X, outcome in test_dataloader:
#         print(X.shape, outcome)

# with open(output, "w") as g:
#     for (filename, pred) in zip(ds.fnames, yhat):
#         for (i,p) in enumerate(pred):
#             g.write("{},{},{:.4f}\n".format(filename, i, p))

removed duplicates, 0


In [28]:
kwargs

{'target_type': 'Filename',
 'mean': array([31.834011, 31.95879 , 32.082172]),
 'std': array([48.866325, 49.137333, 49.361984]),
 'length': 32,
 'period': 2}

In [41]:
torch.cuda.empty_cache()

save_all = True
block_size = 1
train = False
n = 0      # number of videos processed

errorLog = "allA4c_ef_output_ERROR_FILES.csv"
regression_output = "allA4c_ef_output.csv"

with open(errorLog, 'w') as errors:
    with open(regression_output, 'w') as g:
        with torch.set_grad_enabled(train):
            with tqdm.tqdm(total=len(test_dataloader)) as pbar:
                for i, (X, outcome) in enumerate(test_dataloader):
                    #try:
                    #print(X.shape, flush = True)

                    #y.append(outcome.numpy())
                    #X = X.to(device)
                    #outcome = outcome.to(device)
                    X = torch.flip(X,[3])
                    #block_size = X.shape[1]
                    print(X.shape[1])
                    average = (len(X.shape) == 6)
                    if average:
                        batch, n_clips, c, f, h, w = X.shape
                        X = X.view(-1, c, f, h, w)

                    if block_size is None:
                        outputs = model(X)
                    else:

                        for j in range(0, X.shape[0], block_size):
                            thisX = X[j:(j + block_size)].to(device)
                            thisOutput = model(thisX).detach().cpu()

                            for k, l in enumerate(thisOutput):
                                #print(outcome[0], (j + k), l)
                                g.write(outcome[0] + "," + str(j + k) + "," + str(l.item()) + "\n")

                    n += X.size(0)

                    pbar.set_postfix_str("{:.2f} ({:.2f})".format( n, i))
                    pbar.update()

                        #print(s1, s2, total)
                    #except:
                    #    print(X.shape, flush = True)
                    #    print("something did not go right with", outcome)
                    #    errors.write(outcome[0] + "\n")

                    torch.cuda.empty_cache()
                    #    print("cache cleared!", torch.cuda.memory_allocated(0))



  0%|          | 0/3 [00:00<?, ?it/s]

63


 33%|███▎      | 1/3 [00:06<00:12,  6.08s/it, 63.00 (0.00)]

122


 67%|██████▋   | 2/3 [00:15<00:07,  7.10s/it, 185.00 (1.00)]

134


100%|██████████| 3/3 [00:25<00:00,  8.08s/it, 319.00 (2.00)]


In [40]:
!nvidia-smi

Mon Jan 25 22:34:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:01:00.0 Off |                  N/A |
| 37%   57C    P2   115W / 350W |  12592MiB / 24263MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:21:00.0 Off |                  N/A |
| 36%   

In [12]:
!pip install torch==1.7.0 torchvision===0.8.1 tqdm==4.50.2

     |████████████████████████████████| 70 kB 822 kB/s eta 0:00:01
ERROR: thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.1 which is incompatible.
ERROR: spacy 2.0.16 has requirement regex==2018.01.10, but you'll have regex 2020.5.14 which is incompatible.
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.31.1
    Uninstalling tqdm-4.31.1:
      Successfully uninstalled tqdm-4.31.1


In [ ]:
# Load already processed, to ignore in dataloader

In [ ]:
#Make a file with name of all files already run

firstrun = "C:\\Datasets\\All A4c Videos from Database Query\\allA4c_ef_output_thirdrun.csv"
unique_firstrun =  "C:\\Datasets\\All A4c Videos from Database Query\\alreadyProcessedFiles-new2.csv"
unique_firstrun_previous =  "C:\\Datasets\\All A4c Videos from Database Query\\alreadyProcessedFiles-new.csv"

unique_fnames = []

with open(unique_firstrun_previous) as previous:
    with open(firstrun, 'r') as f:
        with open(unique_firstrun, 'w') as fout:
            for line in previous.readlines():
                fout.write(line)
                unique_fnames.append(line.strip())
            
            print("previous files loaded")#, unique_fnames)
            
            for line in f.readlines():
                fname = line.split(',')[0]
                if fname not in unique_fnames:
                    unique_fnames.append(fname)
                    fout.write(fname + "\n")

print(len(unique_fnames))
    

In [ ]:
# Previous runs

In [ ]:
save_all = True
block_size = 20
train = False
n = 0      # number of videos processed

errorLog = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output_ERROR_FILES.csv")
regression_output = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output.csv")

with open(errorLog, 'w') as errors:
    with open(regression_output, 'w') as g:
        with torch.set_grad_enabled(train):
            with tqdm.tqdm(total=len(test_dataloader)) as pbar:
                for i, (X, outcome) in enumerate(test_dataloader):
                    try:
                        #print(X.shape, flush = True)

                        #y.append(outcome.numpy())
                        #X = X.to(device)
                        #outcome = outcome.to(device)

                        average = (len(X.shape) == 6)
                        if average:
                            batch, n_clips, c, f, h, w = X.shape
                            X = X.view(-1, c, f, h, w)

                        if block_size is None:
                            outputs = model(X)
                        else:

                            for j in range(0, X.shape[0], block_size):
                                thisX = X[j:(j + block_size)].to(device)
                                thisOutput = model(thisX).detach().cpu()

                                for k, l in enumerate(thisOutput):
                                    #print(outcome[0], (j + k), l)
                                    g.write(outcome[0] + "," + str(j + k) + "," + str(l.item()) + "\n")

                        n += X.size(0)

                        pbar.set_postfix_str("{:.2f} ({:.2f})".format( n, i))
                        pbar.update()

                        #print(s1, s2, total)
                    except:
                        print(X.shape, flush = True)
                        print("something did not go right with", outcome)
                        errors.write(outcome[0] + "\n")

                    torch.cuda.empty_cache()
                    #    print("cache cleared!", torch.cuda.memory_allocated(0))



In [ ]:
torch.cuda.empty_cache()

save_all = True
block_size = 30
train = False
n = 0      # number of videos processed

errorLog = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output_ERROR_FILES.csv")
regression_output = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output.csv")

with open(errorLog, 'w') as errors:
    with open(regression_output, 'w') as g:
        with torch.set_grad_enabled(train):
            with tqdm.tqdm(total=len(test_dataloader)) as pbar:
                for i, (X, outcome) in enumerate(test_dataloader):
                    try:
                        #print(X.shape, flush = True)

                        #y.append(outcome.numpy())
                        #X = X.to(device)
                        #outcome = outcome.to(device)

                        average = (len(X.shape) == 6)
                        if average:
                            batch, n_clips, c, f, h, w = X.shape
                            X = X.view(-1, c, f, h, w)

                        if block_size is None:
                            outputs = model(X)
                        else:

                            for j in range(0, X.shape[0], block_size):
                                thisX = X[j:(j + block_size)].to(device)
                                thisOutput = model(thisX).detach().cpu()

                                for k, l in enumerate(thisOutput):
                                    #print(outcome[0], (j + k), l)
                                    g.write(outcome[0] + "," + str(j + k) + "," + str(l.item()) + "\n")

                        n += X.size(0)

                        pbar.set_postfix_str("{:.2f} ({:.2f})".format( n, i))
                        pbar.update()

                        #print(s1, s2, total)
                    except:
                        print(X.shape, flush = True)
                        print("something did not go right with", outcome)
                        errors.write(outcome[0] + "\n")

                    torch.cuda.empty_cache()
                    #    print("cache cleared!", torch.cuda.memory_allocated(0))

